In [1]:
#Import libraries

import numpy as np
import pandas as pd
import glob
import time

from datetime import datetime, timedelta, date, time

In [2]:
all_data = pd.DataFrame()

for f in glob.glob("../*.xlsx"):
    df = pd.read_excel(f)
    all_data = all_data.append(df)
    
all_data = all_data.drop_duplicates(["id_tweets"])

In [3]:
#Get important variable

data = all_data[['time', 'user', 'mentions']]
data = data.sort_values(by='time').reset_index(drop=True)
data.head()

,time,user,mentions
0,2020-04-30 18:20:13,ulinyusron,[]
1,2020-04-30 18:21:47,hendribudiawan2,['ulinyusron']
2,2020-04-30 18:22:33,kemalarsjad,['ulinyusron']
3,2020-04-30 18:24:44,widhiholic,['ulinyusron']
4,2020-04-30 18:26:38,WeizmanJalmav,['ulinyusron']


In [4]:
#Select only 1 day from first tweet

batas = ((pd.to_datetime(data['time'][0]) + timedelta(1)).strftime("%Y-%m-%d %H:%M:%S"))
data = data[data['time'] <= batas]

In [5]:
#Drop unwanted character
#Not effective

data['mentions_rapih'] = data['mentions'].str.replace("'", '')
data['mentions_rapih'] = data['mentions_rapih'].str.replace("]", '')
data['mentions_rapih'] = data['mentions_rapih'].str.replace("[", '')
data['mentions_rapih'] = data['mentions_rapih'].str.replace(" ", '')

In [6]:
new_data = pd.DataFrame(data['mentions_rapih'].str.split(',').tolist(), index=data['user']).stack()
new_data = new_data.reset_index([0, 'user'])
new_data.columns = ['user','mentions_rapih']
new_data.head()

,user,mentions_rapih
0,ulinyusron,
1,hendribudiawan2,ulinyusron
2,kemalarsjad,ulinyusron
3,widhiholic,ulinyusron
4,WeizmanJalmav,ulinyusron


In [7]:
new_data2 = pd.DataFrame(data['mentions_rapih'].str.split(',').tolist(), index=data['time']).stack()
new_data2 = new_data2.reset_index([0, 'time'])
new_data2.columns = ['time','mentions_rapih']
new_data2.head()

,time,mentions_rapih
0,2020-04-30 18:20:13,
1,2020-04-30 18:21:47,ulinyusron
2,2020-04-30 18:22:33,ulinyusron
3,2020-04-30 18:24:44,ulinyusron
4,2020-04-30 18:26:38,ulinyusron


In [8]:
#Make every combination of interaction

new_data['time'] = new_data2['time']
new_data.head()

,user,mentions_rapih,time
0,ulinyusron,,2020-04-30 18:20:13
1,hendribudiawan2,ulinyusron,2020-04-30 18:21:47
2,kemalarsjad,ulinyusron,2020-04-30 18:22:33
3,widhiholic,ulinyusron,2020-04-30 18:24:44
4,WeizmanJalmav,ulinyusron,2020-04-30 18:26:38


In [9]:
#Change blank mentions_rapih to user name

INDEX_KOSONG = new_data[new_data['mentions_rapih']==''].index.values.tolist()
for i in INDEX_KOSONG:
    new_data.iloc[i][1] = new_data.iloc[i][0]

In [10]:
#Save data frame to excel

new_data.to_excel(r'..\rapih_CidukSaidDidu.xlsx', index=False)